## CPSC 439 (Spring 2022) Project 1: Turing Machines

###  Group Members
* Bradley Diep
* John Dinh
* Jason Duong
* Omid Nikjoo

### Documentation
   * Each transition is represented as a tuple $(a, b, c)$ encoded as $1^a01^b01^c$ for $a, b, c \in N$
   * Each final state, $d$, is represented as $1^d0$ for $d\in N$
   * The $00$ token string deliminates objects T and S

### Example
``10110111001101 = T: (1, 11, 111), S: {11, 1} = T: (q0, q1, q2), S: {q1, q0}``

# Hints
* can use any data structure, but still need {0, 1}*
* treat multiple tape heads as separate variables (more than one working variable)

In [92]:
from collections import namedtuple

# Computational Model: DFA
DFA = namedtuple('DFA', ['T', 'S'])
BLANK, ZERO, ONE = ['.', '0', '1']

def encode(M):
    '''converts DFA model to binary representation'''
    code = ''
    for (current, symbol), successor in M.T.items():
        code += ''.join(list(map(lambda offset: ONE * (offset + 1) + ZERO,
                                 [current, symbol, successor])))
    
    code += ZERO + ''.join(list(map(lambda offset: ONE * (offset + 1), M.S)))
    return code

# DFA Representation
DFA.__repr__ = encode

In [93]:
'''Figure 6.3'''

# States
q0, q1, q2, q3 = list(range(4))

# Transition Function
δ = dict([ 
    ((q0, 0), q0), ((q0, 1), q1),
    ((q1, 0), q1), ((q1, 1), q0),
])

XOR_DFA = DFA(δ, [q1])
print(f'XOR-DFA: {XOR_DFA!r}')

XOR-DFA: 101010101101101101011011011010011


In [94]:
'''Figure 6.4'''

# Transition Function
δ = dict([
    ((q0, 0), q2), ((q0, 1), q3),
    ((q1, 0), q0), ((q1, 1), q3),
    ((q2, 0), q3), ((q2, 1), q1),
    ((q3, 0), q3), ((q3, 1), q3),
])

F_DFA = DFA(δ, [q0])
print(f'F-DFA: {F_DFA!r}')

F-DFA: 101011101011011110110101011011011110111010111101110110110111101011110111101101111001


In [95]:
M = XOR_DFA

alphabet = {0, 1}
state = sorted(list(set(([x[0] for x in M.T.keys()]))))
current = state[0]
final = M.S

print(state, alphabet, current, final, sep=', ')

[0, 1], {0, 1}, 0, [1]


In [132]:
alphabet = ['0', '1', '▷', '🔥']
states = ['Q' + str(i) for i in range(6)] + ['QA', 'QR']
input_string = '00010001'

head, state = 0, states[0]
tape = ['▷'] + [x for x in input_string] + ['🔥']
print(''.join(tape))

▷00010001🔥


In [133]:
class HaltExecution(Exception):
    pass

def debug():
    print(f'{state:10} {tape[head]:5}', end='')

def transition(state, sym):
    move = 'R'
    if state in ['Q0', 'Q1']:
        if sym == '🔥':
            state = 'QA' if state == 'Q1' else 'QR'
            move = 'L'
        elif sym == '1':
            state = 'Q0' if state == 'Q1' else 'Q1'
    else:
        move = "H"
        if state in ['QA', 'QR']:
            if sym != '▷':
                move, sym = 'L', '🔥'
            else:
                move = 'R'
                state = 'RETURN1' if state == 'QA' else 'RETURN0'
        elif state[:-1] == "RETURN":
            move = "H"
            sym = state[-1]
    return state, sym, move
    
def run():
    '''simulates a DFA given an input'''
    global state, head, tape
    try:
        while True:
            debug()
            state, tape[head], action = transition(state, tape[head])
            print(f'{state:10} {tape[head]:5} {action}')

            if action == 'L':
                head = max(0, head-1)
            elif action == 'R':
                head += 1
                if head == len(tape):
                    tape.append('🔥')
            elif action == 'S':
                continue
            elif action == 'H':
                raise HaltExecution('Process Ended')
    except HaltExecution:
        print(tape)
        print(''.join(tape[tape.index('▷')+1 : tape.index('🔥')]))

run()

Q0         ▷    Q0         ▷     R
Q0         0    Q0         0     R
Q0         0    Q0         0     R
Q0         0    Q0         0     R
Q0         1    Q1         1     R
Q1         0    Q1         0     R
Q1         0    Q1         0     R
Q1         0    Q1         0     R
Q1         1    Q0         1     R
Q0         🔥    QR         🔥     L
QR         1    QR         🔥     L
QR         0    QR         🔥     L
QR         0    QR         🔥     L
QR         0    QR         🔥     L
QR         1    QR         🔥     L
QR         0    QR         🔥     L
QR         0    QR         🔥     L
QR         0    QR         🔥     L
QR         ▷    RETURN0    ▷     R
RETURN0    🔥    RETURN0    0     H
['▷', '0', '🔥', '🔥', '🔥', '🔥', '🔥', '🔥', '🔥', '🔥']
0
